# Multi Train Gradient Update

## Importing Libraries

In [1]:
from typing import Dict
import threading

import gym
import numpy as np
import torch

from stable_baselines3 import PPO as ALGO
from stable_baselines3.common.evaluation import evaluate_policy

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [2]:
# Hyper-Parameters
NUM_CLIENT_MODELS = 8
NUM_TRAINING_STEPS = 1000
NUM_ITERATIONS = 10
ENV_NAME = 'CartPole-v1'


## Init. ENV and Model

In [3]:
env = gym.make(ENV_NAME)
global_model = ALGO(
    "MlpPolicy",
    env
)

client_models = [ALGO("MlpPolicy", gym.make(ENV_NAME)) for i in range(NUM_CLIENT_MODELS)]

## Functions to Evaluate Model and Train Model within Thread

In [4]:
def evaluate(models, env, index, eval_results, message = '', verbose = 0):
    # print('Starting Eval')
    fitnesses = []
    iterations = 10
    for i in range(iterations):
        fitness, _ = evaluate_policy(models[index], env)
        if verbose == 1:
            print(i, fitness, end=" ")
        fitnesses.append(fitness)

    mean_fitness = np.mean(sorted(fitnesses))
    eval_results[index] = mean_fitness
    print(f'Type {message} Mean reward: {mean_fitness}')

In [5]:
def train(models, index, timesteps):
    # print('Starting Training')
    models[index] = models[index].learn(reset_num_timesteps=False, total_timesteps=timesteps)
    # print('Completed Training')


In [6]:
def multithread_eval(client_models):
    # Create Threads
    client_threads = []
    eval_results = [0 for i in range(len(client_models))] 
    for ci in range(NUM_CLIENT_MODELS):
        thread = threading.Thread(target=evaluate, args=(client_models, gym.make(ENV_NAME), ci, eval_results, f'Trained Model {ci}'))
        client_threads.append(thread)

    # Start Threads
    for thread in client_threads:
        thread.start()

    # Join Threads (wait until thread is completely executed)
    for thread in client_threads:
        thread.join()

    return eval_results


## Initial Evaluation

In [7]:
for model in client_models:
    model.set_parameters(global_model.get_parameters())

global_model.save('initial')

evaluate([global_model], env, 0, [0])

multithread_eval(client_models)

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Type  Mean reward: 9.56
Type Trained Model 7 Mean reward: 9.56
Type Trained Model 0 Mean reward: 9.55
Type Trained Model 4 Mean reward: 9.500000000000002
Type Trained Model 2 Mean reward: 9.520000000000001
Type Trained Model 6 Mean reward: 9.620000000000001
Type Trained Model 3 Mean reward: 9.56
Type Trained Model 1 Mean reward: 9.55
Type Trained Model 5 Mean reward: 9.549999999999999


[9.55,
 9.55,
 9.520000000000001,
 9.56,
 9.500000000000002,
 9.549999999999999,
 9.620000000000001,
 9.56]

# Train and Evaluate

In [8]:
# Evaluation Before Iterated Training
evaluate([global_model], env, 0, [0], "Global Initial Model")

for i in range(NUM_ITERATIONS):
    print('Train Iter: ', i)

    # Create Threads
    client_threads = [] 
    for ci in range(NUM_CLIENT_MODELS):
        thread = threading.Thread(target=train, args=(client_models, ci, NUM_TRAINING_STEPS))
        client_threads.append(thread)


    # Start Threads
    for thread in client_threads:
        thread.start()

    # Join Threads (wait until thread is completely executed)
    for thread in client_threads:
        thread.join()

    # Optimization Steps Check
    print('Optim Steps: ', client_models[0].get_parameters()['policy.optimizer']['state'][0]['step'])

    # Evaluation after Training
    results = multithread_eval(client_models)
    total = sum(results)

    # Accumulate Client Parameters / Weights
    global_dict = global_model.policy.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].policy.state_dict()[k].float()*(NUM_CLIENT_MODELS*results[i]/total) for i in range(len(client_models))], 0).mean(0)

    # Load New Parameters to Global Model
    global_model.policy.load_state_dict(global_dict)

    # Load New Parameters to clients
    for model in client_models:
        model.policy.load_state_dict(global_model.policy.state_dict())

    # Evaluate Updated Global Model
    evaluate([global_model], env, 0, [0], 'Global Updated Model', verbose = 0)


Type Global Initial Model Mean reward: 9.73
Train Iter:  0
Optim Steps:  320
Type Trained Model 2 Mean reward: 97.46
Type Trained Model 3 Mean reward: 114.1
Type Trained Model 4 Mean reward: 165.3
Type Trained Model 1 Mean reward: 171.14000000000001
Type Trained Model 6 Mean reward: 279.01
Type Trained Model 5 Mean reward: 294.56
Type Trained Model 7 Mean reward: 303.54
Type Trained Model 0 Mean reward: 313.15999999999997
Type Global Updated Model Mean reward: 319.67
Train Iter:  1
Optim Steps:  640
Type Trained Model 4 Mean reward: 168.42
Type Trained Model 2 Mean reward: 262.83
Type Trained Model 7 Mean reward: 280.82000000000005
Type Trained Model 1 Mean reward: 302.33
Type Trained Model 6 Mean reward: 325.00999999999993
Type Trained Model 5 Mean reward: 335.72
Type Trained Model 3 Mean reward: 348.3
Type Trained Model 0 Mean reward: 353.59
Type Global Updated Model Mean reward: 345.53999999999996
Train Iter:  2
Optim Steps:  960
Type Trained Model 5 Mean reward: 278.76000000000005


In [ ]:
global_model.get_parameters()

In [ ]:
global_model.save('a2c_lunar_multiproc')

In [ ]:
# Exporting Params as JSON
## Function to Convert Params Dict to Flattened List
def flatten_list(params):
    """
    :param params: (dict)
    :return: (np.ndarray)
    """
    params_ = {}
    for key in params.keys():
        params_[key] = params[key].tolist()
    return params_
## Write Parameters to JSON File
import json

all_params = global_model.get_parameters()
pol_params = flatten_list(all_params['policy'])

all_params['policy'] = pol_params

with open('a2c_lunar_multiproc.json', 'w') as f:
    json.dump(all_params, f, indent='\t')

In [ ]:
# model_loaded = ALGO(
#     "MlpPolicy",
#     env
# )

# evaluate(model_loaded,env, verbose=1)

# import json
# with open('a2c_lunar_multiproc.json', 'w') as f:
#     new_params = json.load(f)

# loaded_pol_params = new_params['policy']
# for key in loaded_pol_params.keys():
#     loaded_pol_params[key] = th.tensor(loaded_pol_params[key])

# new_params['policy'] = loaded_pol_params

# model_loaded.set_parameters(new_params)

model_loaded = ALGO.load('a2c_lunar_multiproc', env)

In [ ]:
env.reset()
evaluate(model_loaded,env, verbose=1)